In [18]:
# pydantic library used for data validation and settings management using Python type annotations.
from pydantic import BaseModel,Field


In [ ]:
#1. model with mandatory fields
class Person(BaseModel):
    name: str
    age: int
    city: str

person = Person(name="Kirti", age=30, city="DEHRADUN")
print(person)

name='Kirti' age=30 city='DEHRADUN'


In [3]:
from dataclasses import dataclass

@dataclass
class Car:
    make: str
    model: str
    year: int
car = Car(make="Toyota", model="Camry", year=2020)
print(car)

Car(make='Toyota', model='Camry', year=2020)


In [ ]:
car2 = Car(make="TATA", model = 12 , year=2025) #not getting error if integer is passed instead of string, no validation in dataclass
print(car2)

Car(make='TATA', model=12, year=2025)


In [ ]:
person2 = Person(name="Kirti", age=30, city=31) #getting error if integer is passed instead of string
print(person2)

ValidationError: 1 validation error for Person
city
  Input should be a valid string [type=string_type, input_value=31, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [ ]:
#2. pydantic model with optional fields
from typing import Optional

In [8]:
class Employee(BaseModel):
    id : int
    name: str
    department: str
    salary: Optional[float] = None #optional with default value which is None
    city: Optional[str] = "Unknown"

In [ ]:
# examples with and without optional fields
emp1 = Employee(id=1, name="Alice", department="HR", salary=50000.0) # type casting also done
print(emp1)
emp2 = Employee(id=2, name="Bob", department="IT")
print(emp2)

id=1 name='Alice' department='HR' salary=50000.0 city='Unknown'
id=2 name='Bob' department='IT' salary=None city='Unknown'


In [ ]:
#3. pydantic using base model with list
from typing import List

class Classroom(BaseModel):
    room_number: str
    students: List[str] # list of strings
    capacity: int

In [12]:
#create a classroom
classroom = Classroom(room_number="101A", students=["Alice", "Bob", "Charlie"], capacity=30)
print(classroom)

room_number='101A' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [13]:
classroom = Classroom(room_number="101A", students=("Alice", "Bob", "Charlie"), capacity=30)
print(classroom)

room_number='101A' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [14]:
classroom = Classroom(room_number="101A", students={"Alice", "Bob", "Charlie"}, capacity=30)
print(classroom)

room_number='101A' students=['Charlie', 'Alice', 'Bob'] capacity=30


In [16]:
try:
    classroom = Classroom(room_number="101A", students=["Alice", 123, "Charlie"], capacity=30)
    print(classroom)
except Exception as e:
    print("Error:", e)

Error: 1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


In [ ]:
## 3. implement pydantic model  with nested models in langchain
from pydantic import Field

In [17]:
class Address(BaseModel):
    street: str
    city: str
    zip_code: str

class Customer(BaseModel):
    customer_id: int
    name: str
    email: str
    address: Address # nested model

# create an instance of Customer with nested Address
customer = Customer(
    customer_id=1,
    name="Kirti",
    email="abc@gmail.com",
    address=Address(
        street="123 Main St",
        city="Dehradun",
        zip_code="12345"
    )
)
print(customer)

customer_id=1 name='Kirti' email='abc@gmail.com' address=Address(street='123 Main St', city='Dehradun', zip_code='12345')


In [ ]:
# pydantic fields: constraints and Customization    
# the field function in pydantic enhances model fields with additional validation and metadata.
# It allows setting constraints (like min/max length, value ranges), default values, aliases, and descriptions.
# This ensures data integrity and provides clear documentation for each field in the model.

class Product(BaseModel):
    product_id: int
    name: str = Field(..., min_length=3, max_length=50) # mandatory field with constraints
    price: float = Field(gt=0,le=1000) # price must be greater than 0
    description: Optional[str] = Field(None, max_length=200) # optional field with max length constraint 
    quantity: int = Field(1, ge=0) # default value is 1 and quantity must be non-negative

# create a product instance
product = Product(product_id=101, name="Laptop", price=999.99, description="A high-end laptop", quantity=10)
print(product)   

product_id=101 name='Laptop' price=999.99 description='A high-end laptop' quantity=10


In [21]:
class User(BaseModel):
    id: int
    name: str =Field(..., min_length=2, max_length=30, description="Unique user name must be between 2 and 30 characters")
    friends: List[int] = []  # default empty list
    age: Optional[int] = Field(default=18, ge=0, le=120, description="Age must be between 0 and 120") # age is optional but if provided must be between 0 and 120

In [22]:
# create a user instance
user = User(id=1, name="Alice", friends=[2, 3], age=25)
print(user)
user2 = User(id=2, name="B", friends=[1]) #name too short, age default to 18
print(user2)

id=1 name='Alice' friends=[2, 3] age=25


ValidationError: 1 validation error for User
name
  String should have at least 2 characters [type=string_too_short, input_value='B', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short

In [27]:
print(user.model_dump_json())

{"id":1,"name":"Alice","friends":[2,3],"age":25}


In [26]:
print(user.model_json_schema())

{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'description': 'Unique user name must be between 2 and 30 characters', 'maxLength': 30, 'minLength': 2, 'title': 'Name', 'type': 'string'}, 'friends': {'default': [], 'items': {'type': 'integer'}, 'title': 'Friends', 'type': 'array'}, 'age': {'anyOf': [{'maximum': 120, 'minimum': 0, 'type': 'integer'}, {'type': 'null'}], 'default': 18, 'description': 'Age must be between 0 and 120', 'title': 'Age'}}, 'required': ['id', 'name'], 'title': 'User', 'type': 'object'}
